# 🚗 Vehicle Speed Detection using YOLOv8

This notebook detects vehicles in a video, tracks them across frames,
and estimates their speed using perspective transformation and pixel displacement.

### Key Features
- YOLOv8 object detection
- ByteTrack multi-object tracking
- Perspective transform for real-world mapping
- Speed estimation in km/h
- Output annotated video

This notebook is designed to be **modular, readable, and production-ready**.


## ⚡ Pro Tip: Enable GPU Acceleration

If you are running this notebook in **Google Colab**:

1. Go to **Edit → Notebook settings**
2. Set **Hardware accelerator** to **GPU**
3. Click **Save**

GPU acceleration significantly improves inference speed.


In [ ]:
!nvidia-smi


/bin/bash: line 1: nvidia-smi: command not found


## 📦 Install Required Libraries

We install all required dependencies:
- `ultralytics` for YOLOv8
- `supervision` for tracking and annotation
- `opencv-python` for video processing
- `tqdm` for progress visualization


In [ ]:
!pip install -U ultralytics supervision opencv-python tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.12.0.88
    Uninstalling opencv-python-4.12.0.88:
      Successfully uninstalled opencv-python-4.12.0.88


## 📚 Import Libraries

This section imports all Python libraries used for:
- Deep learning
- Video processing
- Tracking
- Numerical computation


In [ ]:
from ultralytics import YOLO
import supervision as sv
import cv2
import numpy as np
from tqdm import tqdm
from collections import defaultdict, deque


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## ⚙️ Configuration Parameters

These parameters control:
- Model selection
- Detection thresholds
- Input/output paths
- Speed estimation constraints


In [ ]:
MODEL_NAME = "yolov8n.pt"
MODEL_RESOLUTION = 1280
CONFIDENCE_THRESHOLD = 0.3
IOU_THRESHOLD = 0.7

SOURCE_VIDEO_PATH = "/content/input.mp4"
TARGET_VIDEO_PATH = "/content/output.mp4"

SPEED_LIMIT = 60  # km/h (used later)


## 🎯 Load YOLO Model and Video Information

- Loads YOLOv8 model
- Extracts video metadata
- Creates frame generator
- Initializes ByteTrack tracker


In [ ]:
model = YOLO(MODEL_NAME)

video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

frame_generator = sv.get_video_frames_generator(
    source_path=SOURCE_VIDEO_PATH
)

byte_track = sv.ByteTrack(frame_rate=video_info.fps)


## 📐 Perspective Transformation

This class converts image coordinates into a top-down view,
allowing more accurate distance and speed calculation.


In [ ]:
class ViewTransformer:
    def __init__(self, source: np.ndarray, target: np.ndarray):
        source = source.astype(np.float32)
        target = target.astype(np.float32)
        self.m = cv2.getPerspectiveTransform(source, target)

    def transform_points(self, points: np.ndarray) -> np.ndarray:
        if points.size == 0:
            return points
        reshaped = points.reshape(-1, 1, 2).astype(np.float32)
        transformed = cv2.perspectiveTransform(reshaped, self.m)
        return transformed.reshape(-1, 2)


## 📍 Define Source and Target Regions

These points define:
- Source: road plane in camera view
- Target: bird's-eye mapped plane


In [ ]:
SOURCE = np.array([
    [400, 720],
    [880, 720],
    [1200, 200],
    [80, 200]
])

TARGET = np.array([
    [0, 720],
    [400, 720],
    [400, 0],
    [0, 0]
])

view_transformer = ViewTransformer(
    source=SOURCE,
    target=TARGET
)


## 🖊️ Annotation Configuration

Initializes bounding boxes, labels, and trace drawing
with resolution-adaptive scaling.


In [ ]:
width, height = video_info.resolution_wh

thickness = max(1, int((width + height) / 1000))
text_scale = max(0.4, (width + height) / 3000)

box_annotator = sv.BoxAnnotator(thickness=thickness)

label_annotator = sv.LabelAnnotator(
    text_scale=text_scale,
    text_thickness=thickness,
    text_position=sv.Position.BOTTOM_CENTER
)

trace_annotator = sv.TraceAnnotator(
    thickness=thickness,
    trace_length=int(video_info.fps * 2),
    position=sv.Position.BOTTOM_CENTER
)


## 🧮 Speed Estimation Setup

Tracks Y-coordinate displacement across frames
and converts pixel movement into km/h.


In [ ]:
coordinates = defaultdict(
    lambda: deque(maxlen=int(video_info.fps))
)

PIXELS_PER_METER = 20  # tune once


## 🚀 Process Video Frames

For each frame:
- Detect vehicles
- Track objects
- Transform coordinates
- Estimate speed
- Annotate frame
- Save output


In [ ]:
with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame in tqdm(frame_generator, total=video_info.total_frames):

        result = model(frame, imgsz=MODEL_RESOLUTION, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(result)

        detections = detections[detections.confidence > CONFIDENCE_THRESHOLD]
        detections = detections[detections.class_id != 0]
        detections = detections.with_nms(IOU_THRESHOLD)
        detections = byte_track.update_with_detections(detections)

        points = detections.get_anchors_coordinates(
            anchor=sv.Position.BOTTOM_CENTER
        )

        points = view_transformer.transform_points(points).astype(int)

        labels = [""] * len(detections)

        WINDOW = int(video_info.fps * 0.5)
        MAX_PIXEL_JUMP = 25

        for i, (tracker_id, point) in enumerate(
            zip(detections.tracker_id, points)
        ):
            y = point[1]

            if coordinates[tracker_id]:
                last_y = coordinates[tracker_id][-1]
                y = last_y + np.clip(
                    y - last_y,
                    -MAX_PIXEL_JUMP,
                    MAX_PIXEL_JUMP
                )

            coordinates[tracker_id].append(y)

            if len(coordinates[tracker_id]) < WINDOW:
                continue

            y_values = list(coordinates[tracker_id])[-WINDOW:]
            pixel_distance = abs(y_values[-1] - y_values[0])

            time_seconds = WINDOW / video_info.fps
            meters = pixel_distance / PIXELS_PER_METER
            speed_kmh = (meters / time_seconds) * 3.6

            if 3 <= speed_kmh <= 140:
                labels[i] = f"{int(speed_kmh)} km/h"

        frame = trace_annotator.annotate(frame, detections)
        frame = box_annotator.annotate(frame, detections)
        frame = label_annotator.annotate(frame, detections, labels)

        sink.write_frame(frame)


100%|██████████| 722/722 [07:36<00:00,  1.58it/s]


## ✅ Processing Complete

The output video has been successfully generated
with speed annotations.


In [ ]:
print("✅ DONE. Output saved at:", TARGET_VIDEO_PATH)
